In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geohash

In [21]:
featuredDataset = pd.read_csv('featured-dataset_new_KNN.csv')
featuredDataset.head()

,Unnamed: 0,Unnamed: 0.1,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,...,day_cos,day_sin,weekend,x_start,y_start,z_start,location_start,location_end,end_lat,end_lon
0,0,0,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,...,0.157050,-0.987591,1,-0.055090,0.712922,0.699076,wqp25w569,wqp25tkvt,33.779811,111.605885
1,1,1,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,...,0.789543,0.613695,0,0.746854,-0.615984,-0.250546,ww4nj3h7m,ww4nj3u7r,34.814875,115.549374
2,2,2,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,...,0.116918,0.993142,0,0.762286,-0.641939,-0.082670,ww4jj4hfq,ww4nj9e9j,34.813136,115.559243
3,3,3,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,...,0.023061,0.999734,0,0.740919,-0.620137,-0.257816,ww4nj4rph,ww4nj9e9h,34.813141,115.559217
4,4,4,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,...,-0.003740,0.999993,0,0.752481,-0.608090,-0.252980,ww4nj9edj,ww4muu95u,34.786126,115.874361


In [22]:
featuredDataset['mon_num'] = featuredDataset['month'].apply(lambda row: row/12.)
featuredDataset.head()

,Unnamed: 0,Unnamed: 0.1,year,month,day,time_cat,time_num,time_cos,time_sin,day_cat,...,day_sin,weekend,x_start,y_start,z_start,location_start,location_end,end_lat,end_lon,mon_num
0,0,0,2018,1,20,10:0,0.425694,-0.892979,0.450098,Saturday,...,-0.987591,1,-0.055090,0.712922,0.699076,wqp25w569,wqp25tkvt,33.779811,111.605885,0.083333
1,1,1,2018,2,12,17:30,0.736111,-0.087156,-0.996195,Monday,...,0.613695,0,0.746854,-0.615984,-0.250546,ww4nj3h7m,ww4nj3u7r,34.814875,115.549374,0.166667
2,2,2,2018,2,13,14:45,0.619444,-0.731354,-0.681998,Tuesday,...,0.993142,0,0.762286,-0.641939,-0.082670,ww4jj4hfq,ww4nj9e9j,34.813136,115.559243,0.166667
3,3,3,2018,2,13,17:15,0.724306,-0.160743,-0.986996,Tuesday,...,0.999734,0,0.740919,-0.620137,-0.257816,ww4nj4rph,ww4nj9e9h,34.813141,115.559217,0.166667
4,4,4,2018,2,13,18:0,0.754167,0.026177,-0.999657,Tuesday,...,0.999993,0,0.752481,-0.608090,-0.252980,ww4nj9edj,ww4muu95u,34.786126,115.874361,0.166667


In [23]:
columns_all_features = featuredDataset.columns
columns_X = ['day_num', 'mon_num', 'x_start', 'y_start', 'z_start']
columns_y = ['end_lat', 'end_lon']
X = featuredDataset[columns_X]
y = featuredDataset[columns_y]

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)# test_size：0.05

In [25]:
print ('X: ({}, {})'.format(*X.shape))
print ('y: ({}, {})'.format(*y.shape))
print ('X_train: ({}, {})'.format(*X_train.shape))
print ('y_train: ({}, {})'.format(*y_train.shape))
print ('X_test: ({}, {})'.format(*X_test.shape))
print ('y_test: ({}, {})'.format(*y_test.shape))

X: (1495814, 5)
y: (1495814, 2)
X_train: (1421023, 5)
y_train: (1421023, 2)
X_test: (74791, 5)
y_test: (74791, 2)


In [26]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [27]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X,y)
    print ("BEST", gs.best_params_, gs.best_score_, gs.cv_results_)
    best = gs.best_estimator_
    return best

In [28]:
# Create a k-Nearest Neighbors Regression estimator
knn_estimator = KNeighborsRegressor()
#knn_parameters = {"n_neighbors": [1,2,5,10,20,50,100]}
knn_parameters = {"n_neighbors": [3, 4, 5]}
knn_best = cv_optimize(knn_estimator, knn_parameters, X_train, y_train, score_func='neg_mean_squared_error')

BEST {'n_neighbors': 3} -19.2106313342 {'mean_fit_time': array([ 1.28449655,  1.27426653,  1.27279406]), 'std_fit_time': array([ 0.02442379,  0.01739481,  0.01864014]), 'mean_score_time': array([ 4.42137208,  5.0237824 ,  5.47929144]), 'std_score_time': array([ 0.01878866,  0.05210238,  0.01564817]), 'param_n_neighbors': masked_array(data = [3 4 5],
             mask = [False False False],
       fill_value = ?)
, 'params': [{'n_neighbors': 3}, {'n_neighbors': 4}, {'n_neighbors': 5}], 'split0_test_score': array([-19.15098948, -19.60008552, -20.11616158]), 'split1_test_score': array([-19.21519632, -19.61804741, -20.15151665]), 'split2_test_score': array([-19.22686243, -19.75222302, -20.21399632]), 'split3_test_score': array([-19.26701859, -19.75352486, -20.25518128]), 'split4_test_score': array([-19.19308998, -19.69089273, -20.20839927]), 'mean_test_score': array([-19.21063133, -19.68295465, -20.18905096]), 'std_test_score': array([ 0.03828487,  0.06468621,  0.04917628]), 'rank_test_sco

In [29]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
knn_reg=knn_best.fit(X_train, y_train)
knn_training_accuracy = knn_reg.score(X_train, y_train)
knn_test_accuracy = knn_reg.score(X_test, y_test)
print ("############# based on standard predict ################")
print ("R^2 on training data: %0.8f" % (knn_training_accuracy))
print ("R^2 on test data:     %0.8f" % (knn_test_accuracy))

############# based on standard predict ################
R^2 on training data: 0.85524373
R^2 on test data:     0.67259719


In [ ]:
# Calculate the Root Mean Squared Error
np.sqrt(mean_squared_error(knn_reg.predict(X_test),y_test))

In [ ]:
sampleds = pd.DataFrame(featuredDataset, columns=(columns_X + columns_y))
sampleds = sampleds.sample(10)
sampleds

In [ ]:
y_pred = knn_reg.predict(sampleds.iloc[:,:-2])
y_pred

In [ ]:
from sklearn.externals import joblib
joblib.dump(knn_reg, 'k_nearest_model_11062304.pkl')